

# White Dwarfs Runge Kutta 4








## Importing Libraries

In [13]:
import math
import numpy as np
import matplotlib.ticker as ticker
import pandas as pd
import csv
from decimal import Decimal

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


import matplotlib.colors as mcolors

import matplotlib.pyplot as plt

## Plot Class


In [5]:


class Plot:
    
    def __init__(self, dataframes, x_col, y_col, oneStar=True, rho_c = 0):   #Defines object fig axis for plotting
        """Inicializa la figura con los DataFrames y columnas X e Y."""
        self.dataframes = dataframes  # Lista de DataFrames
        self.x_col = x_col  # Columna para eje X
        self.y_col = y_col  # Columna para eje Y
        
        self.rho_c = rho_c
        self.oneStar = oneStar
        
        # Inicializa la figura y el eje
        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.colors = ['black', 'blue', 'orange', 'green', 'red', 'purple', 'brown', 'magenta', 'gray', 'goldenrod', 'deepskyblue']  
        self.k_vals = [round(x * 0.1, 1) for x in range(-5, 6)]
        
        # Graficar las líneas para cada DataFrame en la figura inicial
        for i in range(len(self.k_vals)):
            sns.lineplot(data=self.dataframes[i], x=self.x_col, y=self.y_col, color=self.colors[i], 
                         label="k={}".format(self.k_vals[i]))
            

    def set_ticks(self, major_ticks, minor_ticks):          #Sets ticks minor and major for both axis
        """Configura los ticks mayores y menores para el eje."""
        self.ax.xaxis.set_major_locator(ticker.MultipleLocator(major_ticks[0]))
        self.ax.yaxis.set_major_locator(ticker.MultipleLocator(major_ticks[1]))

        self.ax.xaxis.set_minor_locator(ticker.MultipleLocator(minor_ticks[0]))
        self.ax.yaxis.set_minor_locator(ticker.MultipleLocator(minor_ticks[1]))
        
        self.ax.tick_params(axis='both', which='major', direction='in', top=True, right=True)
        self.ax.tick_params(axis='both', which='minor', direction='in', top=True, right=True)
            
    def set_labels(self, x_label, y_label, fontsize=12):    #Sets labels for x and y axis and the fontsize
        """Configura las etiquetas de los ejes."""
        self.ax.set_xlabel(x_label, fontsize = fontsize)
        self.ax.set_ylabel(y_label,  fontsize = fontsize )
        
         
    def set_limits(self, y_limits ):                # Set axis limits
        
        self.ax.set_ylim(y_limits[0], y_limits[1])
        
    def set_legend(self,size,loc='upper right'):    # Set Legend size and location
        
        self.ax.legend(frameon = False, fontsize = size,loc= loc)
          
    def show(self):                 #Plots and saves the figure
         

        
        if(self.oneStar):
                plt.savefig("valsVars_{}_{}_eos_{}_ani{}_rhoCent_{:.0E}_oneStar".format(self.x_col,self.y_col, eos , ani, self.rho_c ), format='pdf')  # Guarda la figura como PDF
        else:

                plt.savefig("valsVars_{}_{}_eos_{}_ani{}_sevStars".format(self.x_col,self.y_col, eos , ani ), format='pdf')  # Guarda la figura como PDF

        
        plt.show() 

    



## Maximum Finder Function

In [3]:


def maxFinder(var,eos,ani,k_vals):
    
    ind_max_masses=[]  #List to save the row indeces of max masses
    
    var_max_masses= []   #List to save the value of the desired variable in that index
  



    for  i in range(11):   #Iterate over all values of k to find theirs maximums

      
        k = k_vals[i]
        
        #Define data frames for each k
              
        df = pd.read_csv("valsVars_eos{}_ani{}_sevStars_k_{}.txt".format(eos,ani,k), sep=" ", header=None, names=["rad", "preCent", "preSurf", "eneCentSi", "eneCentMev"  , "eneSurf", "mass", "rShift","ani","aniSca","gravSurf"])

        ind_max_masses.append( df[['mass']].idxmax() ) #Appends an index for a k value
        
        

        var_max_masses.append(df[var][ind_max_masses[i]]) #Appends  value for the index just added
        

        
        
        
         
    return var_max_masses  #Retrieves the list of maximum values for that variable for all k vals

## Differential Equations and variables

In [2]:
# Pressure equation

def dpdr(r, p, m, rho, sig):


    return -( p + rho )*(4*math.pi*r*p*1.3234e-6  + 1.4766*m/(r**2))*((1-2*1.4766*m/r)**(-1)) + 2*sig/r


# Mass equation

def dmdr(r, rho):

    return 4*math.pi*(r**2)*rho/(1.116e6)

# Equation of state BPS for Energy

def eneDenBPS(p):

    a= -15.8306
    b= 11.2974
    c = 0.00664824
    d = 16.9824

    return                  (10**(a + b*math.sqrt( (( d + math.log10((197.33**3)*p) )**2)*c  + 1 )))/(197.33**3)

# Equation of state Polytropic for Energy

def eneDenChandra(p):

    o = 1.47518*(1e-3)
    
   


    return                  (((197.33**3)*p/o)**(3/4))/(197.33**3)


# Equation of state BPS for Pressure

def preBPS(rho):

    a= -15.8306
    b= 11.2974
    c = 0.00664824
    d = 16.9824


    return                 (10**( math.sqrt( ( ((math.log10((197.33**3)*rho) - a)/b)**2  - 1 )/c ) - d ))/(197.33**3)


# Equation of state Polytropic for Pressure

def preChandra(rho):



    o = 1.47518*(1e-3)
    
   


    return                  (o*(rho*(197.33**3))**(4/3))/(197.33**3)


# Anisotropic factor complicated


def aniComp(r, p, m, rho, k):


    return (1.3234e-6)*(k)*(rho+p)*(rho + 3*p)*(r**2)*(1-2*1.4766*m/r)**(-1)

# Anisotropic factor simple


def aniSimp(r,p,m,k):


    return 227.6*k*p*(1-  ( 1 - 2*1.4766*m/r ) )



# Redshift


def redShift(R,M):

    return (1-2*M/R)**-1 - 1



#Surface Gravity



def gravSurf(R,M):
    
    a= (1.989e24)*(2.9979**2)*7.4237e-12
    
    b= (1-2*1.4766*M/R)**(0.5)

    return a*M/(b*(R**2))





## OneStarFunction

In [1]:
# RungeKutta Algorithm




def rungeKutta(r0, rho_0, m0, k, h , cut, oneStar, eos, ani , sig0 = 0):
    
    
     
      
    
    
    
   #Initial values(rho comes in SI, but it's transformed into MeV)
    
    
    rho =rho_0/(1.78266e15)
    
    m = m0
    
    #/(1.78266e15)
    
    
    if(eos== "Chandra"):
        
        
        p= preChandra(rho_0/(1.78266e15))
        
        p0=preChandra(rho_0/(1.78266e15))
      
        
    elif (eos == "BPS"):      
        
            
        p = preBPS(rho_0/(1.78266e15))
        
        p0= preBPS(rho_0/(1.78266e15))
         
    
    
    
    #Creating a file for one stars and writing the initial values
        
    if(oneStar): 
    

        f = open("valsVars_eos{}_ani{}_rhoCent_{:.0E}_k_{}.txt".format(eos, ani ,rho_0,k), "w")
        f.write("{} {} {} {} {} {} {} {}\n".format(r0,p/p0, p,m,rho/(rho_0/(1.78266e15)),rho,sig0/p0, sig0))

    
      
    
    
    while (p/p0 > cut):      
        
        
       

        "Apply Runge Kutta Formulas to find next value of p and m"

        k1 = h * dpdr(r0, p, m, rho, sig0)

        j1 = h* dmdr(r0, rho)

        k2 = h * dpdr(r0 + 0.5 * h, p + 0.5 * k1, m + 0.5*j1, rho , sig0 )
        j2 = h * dmdr(r0 + 0.5 * h,rho )


        k3= h * dpdr(r0 + 0.5 * h, p + 0.5 * k2, m + 0.5*j2, rho , sig0 )
        j3 = h *dmdr( r0 + 0.5*h, rho)

        k4 = h * dpdr(r0 + h, p + k3, m + j3 , rho , sig0 )
        j4 = h* dmdr(r0 + h, rho )

        
        


        # Update next value of p
        p = p + (1.0 / 6.0)*(k1 + 2 * k2 + 2 * k3 + k4)
        
                      
        
        # Update next value of m
        m = m + (1.0 / 6.0)*(j1 + 2 * j2 + 2 * j3 + j4)
        
        
       

        # Update next value of r    
        r0 = r0 + h

        # Update next value of energy acoording to the EOS
        
        
        if(eos== "Chandra"):
            
            rho = eneDenChandra(p)
            
        elif (eos=="BPS"):
            
            
            rho = eneDenBPS(p)
            
        
        if( ani == "Comp"):            
        
            sig0 = aniComp(r0, p, m,rho, k)
            
            
        elif (ani == "Simp"):
            
            
            sig0= aniSimp(r0, p, m, k)
            
            
        
        # Writing the next row of all variables for one star case         
        
       
       
        if (oneStar): 
        
            f.write("{} {} {} {} {} {} {} {}\n".format(r0,p/p0,p ,m,rho/(rho_0/(1.78266e15)),rho,sig0/p0, sig0 ))
        
    
    #Variables for surface values 
    
    rs = redShift(r0,m)
    
    g = gravSurf(r0,m)
    
    
    
    if (oneStar): 
             
        
        f.close()
    
    return p, rho, m, r0, rs, sig0, sig0/p0, g            #Returns surface value of all this quantities

## SeveralStars Code

In [1]:
## p, rho, m, ........... are surface values

def sevStars(eos,ani,k_vals,cut):
    
    # Initial conditions

    r0 = 1e-5 ####(km)
    m0 = 1e-15 ### (Masa solar)
    
    h = 0.01 ###Step
    


    #Orders of magnitud of energy in SI units

    density_ref_central_vals = [1e11,1e12,1e13,1e14]
    
    density_par_central_vals =[] #all values of density


    j=0

    
    #Creating 541 values between the orders of magnitude

    for rho in density_ref_central_vals [:len(density_ref_central_vals )-1]:


        for i in range(int(density_ref_central_vals[j]),int(density_ref_central_vals[j+1]),int(density_ref_central_vals[j]/10)):

            density_par_central_vals.append(i)

        j+=1

    #density_par_central_vals for SI units

    density_par_central_vals.append(density_par_central_vals[len(density_par_central_vals)-1])
    
    #density central_vals for MeV units

    density_central_vals = [rho/(1.7827*1e15) for rho in density_par_central_vals]

    
    
    
    
    
    #Choosing the EOS
    
    if (eos=="Chandra"):
        pressure_central_vals = [ preChandra(rho) for rho in density_central_vals]
    
    elif (eos=="BPS"):
        pressure_central_vals = [ preBPS(rho) for rho in density_central_vals]

    
    #Iterating over all possible values of anisotropic constant k
    

    for k in k_vals:   


        #Creating a file to save all variables for each star (several stars analysis)

        doc = open("valsVars_eos{}_ani{}_sevStars_k_{}.txt".format(eos,ani,k), "w")


        print("k = ", k)

        i=0        
        
        #Iterating over 541 values of central energy density


        for p_c, rho_c_si, rho_c_mev in zip(pressure_central_vals, density_par_central_vals, density_central_vals):

            #using Runge Kutta function to assing surface values to variables


            p,rho , m, r , rs, sig, sigScale, gravSurf=  rungeKutta(r0, rho_c_si, m0, k, h , cut,False, eos  , ani )


    
            print(i+1,r, p_c , p, rho_c_si, rho_c_mev, rho, m , rs, sig, sigScale, gravSurf)

            i +=1


            # Tabla:
            # Radio total   Presión Central   Presión Superficie  Densidad Central  Densidad Superficial  Masa Total  Redshift


            doc.write("{} {} {} {} {} {} {} {} {} {} {}\n".format(r, p_c , p, rho_c_si, rho_c_mev ,rho, m , rs, sig, sigScale, gravSurf))


        doc.close()
   